In [1]:
%run util.py

In [2]:
# N = 2
# N = 3
N = 4
# N = 5

In [3]:
# mu_0: the initial distribution; 1 x (N**2) 
# mu: the actual stationary distribution; 1 x (N**2)
# mu_1: the estimated stationary distribution 
# P: the new transition matrix (the old transition matrix is Q)
# G_1: the estimate of the gradient
# H_1: the estimate of the Hessian
# U_1: an estimated sample path of the Gaussian random vector U; length 1000

mu_01, mu_1, mu_11, P_11, G_11, H_11, U_11 = ChainGen(N)  # P_11 is the 1st ground truth transition matrix
mu_02, mu_2, mu_12, P_12, G_12, H_12, U_12 = ChainGen(N)  # P_12 is the 2nd ground truth transition matrix
mu_03, mu_3, mu_13, P_13, G_13, H_13, U_13 = ChainGen(N)  # P_13 is the 3rd ground truth transition matrix

mu_04, mu_4, mu_14, P_14, G_14, H_14, U_14 = ChainGen(N)  # P_14 is a different transition matrix for testing purposes

saved_data = [mu_01, mu_1, mu_11, P_11, G_11, H_11, U_11, \
              mu_02, mu_2, mu_12, P_12, G_12, H_12, U_12, \
              mu_03, mu_3, mu_13, P_13, G_13, H_13, U_13, \
              mu_04, mu_4, mu_14, P_14, G_14, H_14, U_14]
zdump(saved_data, 'P_11_12_13_14_%d.pkz'%N)

<string>:372: RuntimeWarning: covariance is not positive-semidefinite.


In [4]:
[mu_01, mu_1, mu_11, P_11, G_11, H_11, U_11, \
              mu_02, mu_2, mu_12, P_12, G_12, H_12, U_12, \
              mu_03, mu_3, mu_13, P_13, G_13, H_13, U_13, \
              mu_04, mu_4, mu_14, P_14, G_14, H_14, U_14] = zload('P_11_12_13_14_%d.pkz'%N)

In [5]:
mu_1, mu_2, mu_3, mu_4

(array([ 0.02292687,  0.03481443,  0.02805362,  0.00270147,  0.01745016,
         0.01582231,  0.07371857,  0.12326868,  0.03220867,  0.04604773,
         0.07274319,  0.09394218,  0.01591069,  0.13357525,  0.07042639,
         0.21638979]),
 array([ 0.08601407,  0.21584195,  0.02106916,  0.003195  ,  0.12417195,
         0.04226691,  0.03446625,  0.10529486,  0.02779148,  0.01804809,
         0.02302178,  0.01781988,  0.08814268,  0.03004302,  0.00812405,
         0.15468887]),
 array([ 0.05163249,  0.01622571,  0.02559833,  0.05048216,  0.03014367,
         0.12280655,  0.13673654,  0.0913893 ,  0.027921  ,  0.06033751,
         0.05188578,  0.07922835,  0.03424153,  0.18170629,  0.00515199,
         0.03451279]),
 array([ 0.08173298,  0.03811788,  0.0652468 ,  0.01469266,  0.02411012,
         0.11800559,  0.15074109,  0.00834456,  0.03067713,  0.06591157,
         0.03515068,  0.14080581,  0.06327008,  0.07916633,  0.02140662,
         0.06262009]))

In [6]:
# n_range = range(20, 50, 10)
# n_range = range(30, 90, 20)
n_range = range(40, 130, 30)
# n_range = range(50, 170, 40)

In [7]:
num_test_sample = 400
beta_list = list(np.arange(0, 0.2, 0.01)[1:-1]) + list(np.arange(0.2, 1.01, 0.05)[:-1])

In [8]:
eta_wc = {}
eta_Sanov = {}
test_sample = {}
for n in n_range:
    # Get sample paths of the Markov chain with length n; 
    # these paths will be the test set
    test_sample[str(n)] = []
    for idx in range(int(num_test_sample/4)):
        test_sample[str(n)].append(chain(mu_1, P_11, n))  
    for idx in range(int(num_test_sample/4)):
        test_sample[str(n)].append(chain(mu_2, P_12, n))  
    for idx in range(int(num_test_sample/4)):
        test_sample[str(n)].append(chain(mu_3, P_13, n))  
    for idx in range(int(num_test_sample/4)):
        test_sample[str(n)].append(chain(mu_4, P_14, n)) 
    # Get thresholds for Hoeffding's test corresponding to sample length n    
    for beta in beta_list:
        key = str(n) + '_' + str(beta)
        G_list = [G_11, G_12, G_13]
        H_list = [H_11, H_12, H_13]
        U_list = [U_11, U_12, U_13]
        eta_1 = HoeffdingRuleMarkovRobust_(beta, G_list, H_list, U_list, n)
        eta_2 =  - log(beta) / n
        eta_wc[key] = 1.8 * eta_1
        eta_Sanov[key] = eta_2
zdump([test_sample, eta_wc, eta_Sanov], 'testSample_threshold_robust_%d_adjusted.pkz'%N)

In [9]:
test_sample, eta_wc, eta_Sanov = zload('testSample_threshold_robust_%d_adjusted.pkz'%N)

In [10]:
from __future__ import division

mu_1 = np.reshape(mu_1, (N, N))
mu_2 = np.reshape(mu_2, (N, N))
mu_3 = np.reshape(mu_3, (N, N))

TPR_wc = {}
FPR_wc = {}
TPR_Sanov = {}
FPR_Sanov = {}

for n in n_range:
    for beta in beta_list:
        TP_wc = 0
        FP_wc = 0
        TP_Sanov = 0
        FP_Sanov = 0
        key = str(n) + '_' + str(beta)
        for idx in range(num_test_sample):
            KL1 = KL_est(test_sample[str(n)][idx], mu_1)
            KL2 = KL_est(test_sample[str(n)][idx], mu_2)
            KL3 = KL_est(test_sample[str(n)][idx], mu_3)
            KL = min([KL1, KL2, KL3])
            if idx > 300 and KL > eta_wc[key]:
                TP_wc += 1
            if idx < 300 and KL > eta_wc[key]:
                FP_wc += 1
            if idx > 300 and KL > eta_Sanov[key]:
                TP_Sanov += 1
            if idx < 300 and KL > eta_Sanov[key]:
                FP_Sanov += 1
        TPR_wc[key] = TP_wc / 100
        FPR_wc[key] = FP_wc / 300
        TPR_Sanov[key] = TP_Sanov / 100
        FPR_Sanov[key] = FP_Sanov / 300
        
zdump([TPR_wc, FPR_wc, TPR_Sanov, FPR_Sanov], 'ROCpt_%d_robust_adjusted.pkz'%N)

In [11]:
TPR_wc, FPR_wc, TPR_Sanov, FPR_Sanov = zload('ROCpt_%d_robust_adjusted.pkz'%N)

In [12]:
TPR_wc_list = {}
FPR_wc_list = {}
TPR_Sanov_list = {}
FPR_Sanov_list = {}
        
for n in n_range:
    TPR_wc_list[str(n)] = []
    FPR_wc_list[str(n)] = []
    TPR_Sanov_list[str(n)] = []
    FPR_Sanov_list[str(n)] = []
    TPR_wc_list[str(n)].append(0)
    FPR_wc_list[str(n)].append(0)
    TPR_Sanov_list[str(n)].append(0)
    FPR_Sanov_list[str(n)].append(0)
    for beta in beta_list:
        key = str(n) + '_' + str(beta)
        TPR_wc_list[str(n)].append(TPR_wc[key])
        FPR_wc_list[str(n)].append(FPR_wc[key])
        TPR_Sanov_list[str(n)].append(TPR_Sanov[key])
        FPR_Sanov_list[str(n)].append(FPR_Sanov[key])

    TPR_wc_list[str(n)].append(1)
    FPR_wc_list[str(n)].append(1)
    TPR_Sanov_list[str(n)].append(1)
    FPR_Sanov_list[str(n)].append(1)

In [13]:
import matplotlib.pyplot as plt
import pylab
from pylab import *

ROC_WC_list = []
ROC_Sanov_list = []
style_list = ['bo-', 'gs-', 'r+-', 'c<-', 'm>-', 'y^-']
idx = 0

pylab.xlim(-0.1, 1.15)
pylab.ylim(-0.1, 1.1)

for n in n_range:
    ROC_WC_list.append(plt.plot(FPR_wc_list[str(n)], TPR_wc_list[str(n)], style_list[idx]))
    ROC_Sanov_list.append(plt.plot(FPR_Sanov_list[str(n)], TPR_Sanov_list[str(n)], style_list[idx+1]))
    idx += 2

legend_list_1 = []
legend_list_2 = []
legend_list_1.append(ROC_WC_list)
legend_list_1.append(ROC_Sanov_list)
legend_list_1 = list(np.array(legend_list_1).ravel())

for n in n_range:
    legend_list_2.append("Use threshold $\eta_n^{wc}$; $n=%d$"%n)
for n in n_range:
    legend_list_2.append("Use threshold $\eta_n^{sv}$; $n=%d$"%n)
plt.legend(legend_list_1, legend_list_2, loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid()

savefig('ROC_%d_robust_adjusted.eps'%N)
plt.show()